# Preparing stuff

In [1]:
import os
import glob
import pandas as pd
import numpy as np

import tensorflow as tf

In [ ]:
# !mkdir /content/OI
!unzip /content/drive/MyDrive/DATASET/oi-mini-furniture-1k.zip -d /content/OI/

In [11]:
subset = [
'Chair',
'Desk',
'Couch',
'Bookcase',
'Table',
'Cupboard',
'Bench',
'Drawer',
'Stool',
'Shelf'
 ]

['Table' 'Chair' 'Desk' 'Cupboard' 'Drawer' 'Shelf' 'Bookcase' 'Couch'
 'Stool' 'Bench']


# Create tfrecord

In [12]:
%%writefile generate_tfrecord.py
"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow.compat.v1 as tf

from PIL import Image
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('image_dir', '', 'Path to images')
FLAGS = flags.FLAGS

def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def float_list_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))


# replace row_label with the name you annotated your images as
# I know this is stupid, but it works.
def class_text_to_int(row_label):
    if row_label == 'Chair':
        return 1
    elif row_label == 'Desk':
        return 2
    elif row_label == 'Couch':
        return 3
    elif row_label == 'Bookcase':
        return 4
    elif row_label == 'Table':
        return 5
    elif row_label == 'Cupboard':
        return 6
    elif row_label == 'Bench':
        return 7
    elif row_label == 'Drawer':
        return 8
    elif row_label == 'Stool':
        return 9
    elif row_label == 'Shelf':
        return 10
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'])
        xmaxs.append(row['xmax'])
        ymins.append(row['ymin'])
        ymaxs.append(row['ymax'])
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(filename),
        'image/source_id': bytes_feature(filename),
        'image/encoded': bytes_feature(encoded_jpg),
        'image/format': bytes_feature(image_format),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text': bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(FLAGS.image_dir)
    csv_input = pd.read_csv(FLAGS.csv_input)
    # We have to fix csv input to match this script
    csv_input.rename(columns={    
        'ImageID': 'filename',
        'Width': 'width',
        'Height': 'height',
        'ClassName': 'class',
        'XMin': 'xmin',
        'YMin': 'ymin',
        'XMax': 'xmax',
        'YMax': 'ymax'
    }, inplace=True)
    examples = csv_input[['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']].copy()
    examples['filename'] = examples['filename']+'.jpg'
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()

Writing generate_tfrecord.py


In [14]:
!mkdir /content/tfrecord

In [15]:
!python generate_tfrecord.py --csv_input=/content/OI/train.csv  --image_dir=/content/OI/train --output_path=/content/tfrecord/train.record

2021-02-03 06:46:25.759029: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
I0203 06:46:27.742593 140079996213120 utils.py:157] NumExpr defaulting to 2 threads.
Successfully created the TFRecords: /content/tfrecord/train.record


In [16]:
!python generate_tfrecord.py --csv_input=/content/OI/validation.csv  --image_dir=/content/OI/validation --output_path=/content/tfrecord/validation.record

2021-02-03 06:47:01.462320: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
I0203 06:47:03.760699 140162952152960 utils.py:157] NumExpr defaulting to 2 threads.
Successfully created the TFRecords: /content/tfrecord/validation.record


In [17]:
!du -sh /content/tfrecord

2.6G	/content/tfrecord


# Create pbtxt file

In [18]:
%%writefile /content/tfrecord/label_map.pbtxt
item {
    id: 1
    name: 'Chair'
}

item {
    id: 2
    name: 'Desk'
}

item {
    id: 3
    name: 'Couch'
}

item {
    id: 4
    name: 'Bookcase'
}

item {
    id: 5
    name: 'Table'
}

item {
    id: 6
    name: 'Cupboard'
}

item {
    id: 7
    name: 'Bench'
}

item {
    id: 8
    name: 'Drawer'
}

item {
    id: 9
    name: 'Stool'
}

item {
    id: 10
    name: 'Shelf'
}

Writing /content/tfrecord/label_map.pbtxt


# Move to drive

In [ ]:
# %%time
# !zip -r /content/oi-furniture-1k-tfrecords.zip /content/dataset

  adding: content/dataset/ (stored 0%)
  adding: content/dataset/label_map.pbtx (deflated 72%)
  adding: content/dataset/validation.record (deflated 0%)
  adding: content/dataset/train.record (deflated 0%)
CPU times: user 313 ms, sys: 59.3 ms, total: 372 ms
Wall time: 2min 28s


In [20]:
%%time
!cp -r /content/tfrecord/. /content/drive/MyDrive/DATASET/mini-furniture-tfrecords/

CPU times: user 120 ms, sys: 30.8 ms, total: 151 ms
Wall time: 52.5 s


# Dump

In [ ]:
df = pd.read_csv('/content/OI/validation.csv')

In [ ]:
df.groupby('ImageID')['ClassName'].count()

ImageID
00173afdc7581c41    1
003775320f1d6937    1
0052ea56ee869426    1
00a36f96e31731c4    1
00f3dbbbf5bdf2cf    2
                   ..
feb70e5c996fcb5e    8
fef54ede1354f96b    3
ff065d9a599624a5    5
fff3ce694bc02a09    1
fff50186c03c8474    1
Name: ClassName, Length: 1182, dtype: int64

In [ ]:
df[df['ImageID']=='feb70e5c996fcb5e'].to_csv('df.csv', index_label=False)

In [ ]:
df[df['ImageID']=='feb70e5c996fcb5e']

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,ClassName,Height,Width
3748,feb70e5c996fcb5e,xclick,/m/05kyg_,1,0.011799,0.805310,0.451327,1.000000,0,0,0,0,0,Chest_of_drawers,683,1024
3749,feb70e5c996fcb5e,xclick,/m/05kyg_,1,0.075221,0.395280,0.000000,0.236726,0,0,0,0,0,Chest_of_drawers,683,1024
3750,feb70e5c996fcb5e,xclick,/m/01s105,1,0.022124,1.000000,0.002212,1.000000,0,1,0,0,0,Cabinetry,683,1024
3751,feb70e5c996fcb5e,xclick,/m/0fqfqc,1,0.023599,0.333333,0.497788,0.931416,0,0,1,0,0,Drawer,683,1024
3752,feb70e5c996fcb5e,xclick,/m/0fqfqc,1,0.530973,0.799410,0.610619,1.000000,0,1,0,0,0,Drawer,683,1024
3753,feb70e5c996fcb5e,xclick,/m/0642b4,1,0.025074,0.797935,0.500000,1.000000,0,0,0,0,0,Cupboard,683,1024
3754,feb70e5c996fcb5e,xclick,/m/0642b4,1,0.069322,0.393805,0.000000,0.241150,0,1,0,0,0,Cupboard,683,1024
3755,feb70e5c996fcb5e,xclick,/m/0b3fp9,1,0.016224,0.805310,0.457965,0.657080,0,0,0,0,0,Countertop,683,1024


In [ ]:
!python generate_tfrecord.py --csv_input=/content/df.csv  --image_dir=/content/OI/validation --output_path=df.record

2021-01-31 12:30:30.313819: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
I0131 12:30:32.292475 140625054713728 utils.py:157] NumExpr defaulting to 2 threads.
Successfully created the TFRecords: /content/df.record


In [ ]:
raw_dataset = tf.data.TFRecordDataset("/content/df.record")
len(list(raw_dataset.as_numpy_iterator()))

1

In [ ]:
for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\001\000H\000H\000\000\377\342\014XICC_PROFILE\000\001\001\000\000\014HLino\002\020\000\000mntrRGB XYZ \007\316\000\002\000\t\000\006\0001\000\000acspMSFT\000\000\000\000IEC sRGB\000\000\000\000\000\000\000\000\000\000\000\001\000\000\366\326\000\001\000\000\000\000\323-HP  \000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\021cprt\000\000\001P\000\000\0003desc\000\000\001\204\000\000\000lwtpt\000\000\001\360\000\000\000\024bkpt\000\000\002\004\000\000\000\024rXYZ\000\000\002\030\000\000\000\024gXYZ\000\000\002,\000\000\000\024bXYZ\000\000\002@\000\000\000\024dmnd\000\000\002T\000\000\000pdmdd\000\000\002\304\000\000\000\210vued\000\000\003L\000\000\000\206view\000\000\003\324\000\000\000$lumi\000\000\003\370\000\000\

In [ ]:
raw_dataset = tf.data.TFRecordDataset("/content/validation.record")

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\001\001\340\001\340\000\000\377\333\000C\000\010\006\006\007\006\005\010\007\007\007\t\t\010\n\014\024\r\014\013\013\014\031\022\023\017\024\035\032\037\036\035\032\034\034 $.\' \",#\034\034(7),01444\037\'9=82<.342\377\333\000C\001\t\t\t\014\013\014\030\r\r\0302!\034!22222222222222222222222222222222222222222222222222\377\300\000\021\010\003\000\004\000\003\001\"\000\002\021\001\003\021\001\377\304\000\034\000\000\002\003\001\001\001\001\000\000\000\000\000\000\000\000\000\001\002\000\003\004\005\006\007\010\377\304\000>\020\000\002\002\001\003\003\003\002\004\005\003\002\006\002\002\003\000\001\002\021\003\004\022!\0051A\006\023\"Qa\0242q\221\007\025#B\2013CR\026S\027$4D\222\241T\261%\301b\202\321\377\304\000\033\001\001\001\001\001\001\001\001\001\000\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\377\304\000&\021\001\001\001\001\000\002\

In [ ]:
features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\003\002\002\002\002\002\003\002\002\002\003\003\003\003\004\006\004\004\004\004\004\010\006\006\005\006\t\010\n\n\t\010\t\t\n\014\017\014\n\013\016\013\t\t\r\021\r\016\017\020\020\021\020\n\014\022\023\022\020\023\017\020\020\020\377\333\000C\001\003\003\003\004\003\004\010\004\004\010\020\013\t\013\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\020\377\300\000\021\010\000\305\001\000\003\001\021\000\002\021\001\003\021\001\377\304\000\035\000\000\001\005\001\001\001\001\000\000\000\000\000\000\000\000\000\004\002\005\006\007\010\003\000\t\001\377\304\000V\020\000\000\004\002\004\007\t\014\007\005\005\010\003\001\000\000\002\003\004\005\000\006\007\022\023\024\025\"#$234BCDRSbcs\202\001\010\026%Tr\203\222\242\262\302\3225d\223\243\263\342\360\021!1E\362\027&AUtQaq\204\224\303\323\3436\205\244\301\377\304\000\033\001\001\000\003\001\001\001\001\000\000\000\000\000\000\000\000\000\000\002\003\004\005\001\006\007\377\304\000(\021\001\000\002\002\002\002\002\002\002\002\003\001\000\000\000\000\000\001\002\003\021\004\022\023!\0051\024\"2A\006\025\026#Qa\377\332\000\014\003\001\000\002\021\003\021\000?\000<\221\304\007`\016\001d\231\234\237\000` \t%R\237)?\361=\350\230<g\245sMvT\330G\241\311\231\354\300\006\0066\316\014\346\271\001\3751\326\205\300\023q\23482\246\243\376\357\342\200A\313\237\0333eR\317\337~X\202e\222\376\226\363uT\230\362:\342l\340\034\211?\311\254=\014M\007\224\2176?\251\375h\300q@~m\000g\374\314\002\000d\007k\017)\200X\327)\331\267\216\232 \017@\370\251/\376\350\007PL\337U \377\000\327:%2\367F\324\016)\2259\037zLA\004u:\257V)\2314%KT\236\247j\377\000\311\354\0101\017\033\316\300\301%\313*~\214s<\217\\\277\302\027\301\017\031\331\333\300w\304\337F>\256\377\000\254\001\237\213\026w\211:h\261\261\316\tSl\326\347\362\327<\237\335G\237o~\200\r\326fM\264\266\020\177Rp\313\367\203\036\365x\360&\325I\266\231ew\241\250d:\205\216|c\341I\227\021\350l\342`\364\323S\032\255\225I\036\233\'\000`\317\364\360\034F\273\202\3761?\257b\001\000=Rm\232\336\'\260\261\217\312S}\3146(\3203\245\340\256k\210\353\262\221\001\330\r\257\274\025J\023\377\000\022\000\234\345.r\346\224\362 \026K\212o*\377\000\267\000x\014\211\202A\000\260\037\000`?\214\000\316N\252S9!My\337\242\t\236\016uT\247\3516\304\'\375\335\257\253\001\304lr2\234\352\354{I\377\000i\356\304\320 rU\347\350\311\274\373\016\270{\277:\003\312X\337[3V\305(O\273\353\211\266\370\340\003\032\247\304\311\263\226#\375\t\326\237$\0020\373gNG\241\037\302\030\002S.lT\2476RA\347\353\265\3003\'\346@\023^ ;\022d\007\206\2576\362r9h\256ewU!8S\366\003r\\\331,\266\036\274\373c2\333\337\316(\246d\352\206<O\024\344\347\374\314\362\010PO\003&\317\333\323\017\257\304\210y\227~<#\000\232\251\245\2616s8Lz\355M\360v~\324<\307\343\301\372X\247\352B\226~\234K\205\210\377\000\364v\006P~\010\256\227\333\333\323KvF\357\214M3l\331\003\310\327\023\2530\257<\037\034n\247\270b\277\245\264\311Kw\235\253/\370~\324]\241*M019\377\000\"#\320\376Xhv\002Ye\317\227A\367\221\000\003\304\216\307v\274\336\210Q\320\330\3005#\243k\322k\323\032S\375\t\326p\010:Q\233\3336W5\336\230\233H\001\257S\203g\220\237\367q\016\301\036\021\252K\264\266\036GS\224\207a]\022dL.\244\001\204\237\365\250\005\333\371Jb\017\353\2408\201+g\005\267A\324\300.\352\347y!2W2\017\274r\304\300\023\343\306\317\244\333\017\353\211\312@\022K\343\027\ns \203\371\023\262~\364\000\016\256)\225\\T\245RA\371m\344\350\t\014\007\240\0200&\362^\246\001u\342a\004\237u\331\262\020\013\033\212\236\024\246\337\256\312{\320\000)\301\212\234\210L\245\260\215O#\000N\003c\340\252]Pu\'|\242\004y\217\322\316\333{\0079\245\260\272\314\304/#\221XO\307\211\357\305Y\257\247\235v\252\3479\265\316fr\360=\263!\226\2618\344g\017+\346G77&%\320\342\361gk\026\215(\005\215\262\303\305\226\353\271c\262\206E\024\315\267b8\263\246\201ME\354X6\353\203\010\217z\265N\025iHT3vLz\224\251\276\304\234\244:\251\234,\3072H\r\227\223\357J~\347)\023\3629\327\242\275xjs\223\324\341\324\271\301\033\312\302}\303A\271\207\221\216\364Z\3624\342\231\361\264\205^\204\342y(\356b\313\026szid2LwX\263%{=\372NP:\246Sav\313\305Z\323\307e\347\346\336\230\270oAh\317\315\277\\x\200<\017\216\177\346g\375\264\007`Ln~Uo\327\022\003=\350\004\034\360\331\302\230\220\365\330\345\373\260\024\010\327]\224\335\225%]\327X\300\030K\313g\225@\030\230\310\002@\\\002\007\000\201\252\316H\272\300=\201R\224\334(\362 \027^\365\265%\274@,\tX\277\310\376\306\003\210\032\270Sc\351\344t+2\205\373P\034F\252f\313\245\301\210W\365\'\177\\\007\2011\335\276\235c\\G\336{\260\035\223<1)MzL\253\320\235\254+\260,x\230\3543\376\253\000\212\360\003\014~2!WC\361\300\022;\325\346\363\277\3570\372B\226\331\206|\230\360\024\266z\245V\344\036\243\"O\353\326\366#\231\311\274\304\266\322\2730\320$\262\245\315\357\t\251\327\374\373\216\304q\357\271}\017\026#m\267,0&lM\373\242\3748\366\354DF\2222\205v\3761\267L\326\264Hw\244\235\307$\337\261L4\247\264G\3233S}\036\246M\3434\331x\257\306\313z2C\333\342d\252n\312\2547\302z3K\342\016\0366;\320\034\236\253\301\227\263\323&T}\305FX\235\362\313\372+\306\216\036Y\264\271\327\246\226sl\334\330\2476J\346E\276\246\307W\224\355G\320R\275\241\212\376\223\366\027\233\325\202[\316AF\247{\3631\342\213\306\240L\326\037\233\021\327|\021\216\363\251\035\233OSv#5\375W\211\203\301\374 ? 9\r\235*o\031\245RA\344(\336zM\336&\353\237\001N\022\255O\224\3009\022\252\365\2656!>\000c\220\266pf\313\003\372\023\240\017L\207\311\237~\330\230\002l\037\177\313-\372b`\006\032\356\rv\260>\001\310\234\347fS\001\330\007\375j\001u\340\027^\001`W\023\013\032\345*\263[\314\003j\004\255\212T\237\204\333-\362\334\260-\n\307\346\342@,llw\233\312e.\250\017\353\255<\314H\005\340g\337\345\217\255K\372\354\231\237\004\0020T\315y\302nm\227r\023\223\221\261\312Z\356\276\010\003\023Lm\2333\342S\310\351\216\311\230Wo\034\002\212\261\376\260\366?iR\324\3611\246R\244\204\255\233\017]i\347\217\340\216??\366\227c\213\372\302\343\357`?\t\313d)\352\375\310\311ON\356\013i\254Y\313\375\251\243m/\246\273\347\320\223\027\246J\230\365JT\330\020F\264\343\265eC\317\n\246\272A{\224\373F\217\212\273\255\214ng\273\036A\326\'\\\321\230a~\275Z\220\374\210y\030\346\rsk\213\024\314\333\263\037\3661\356\366\247\307:`\016\371:6}\223\334\274/lL}\206\375\036kno\'\034\355\006\224\246}B\244\277\242\343\330\247Y`\315\222r\307\245\306\332\344\225\3151\027\226$\'\357:\2333=p\212;\034Zv\207\037.I\307>\317\315\254\362\317\006luC\277fk-=\214H\321e\213jR\227\333.\304]\246\373r7\222N\257\361E2\014\003\034\314\2336l| \3629\033`\031\247\355\304\002\306\271\315.\322\304\177\241;\334\000\203\361\300/\0166p\253\361\035r?\374B\034D\022\231\361\2617\3632\017!F\270\233k3<\372\202\200\244\321\276%U\233~4H:\202\003\261\306f\320\004\243\036m\000bc\356\3334\001\200uU\320\037\327\023\0002\224,J\266\226\313\277\3723\240<\006~\014\304\371\377\000XO\307\001\341\202flMzT\330A\3751\'@\001\341r^\022\230\362=\017\313\000z7V\307=\225Q\007\304\3010\0002m+\222\364\306@<Z*\200\360\010\200xm\005\327e\310\037\000x\006\251V\325`\177\\M\244U\233\365\207\274_\332X\302\233\327)S;\036\2257\010\310\223\333\034q\363~\322\354G\353\013\343\274\362j\376\351.\275&\267\\\235]\215\217IP:|X\313\364\353q-\336V\274\377\000?\322\264\263\3436\307\3066\226\253mI\310\362\206\227\347\210Q\347};Q\304\357\0135\235p\251\016\207\274g\266\273\243\313~\2731\357_H\364\235\377\000\361\237\300\226\227\344\367#\330\244f&\242\010Q\226\2668\223\314\262\363\000\020\324\027\257\036uK\242\335\242\271Z\225{\211\217SHs1\013\371\022Im\273\331s4\207\027\322w(\370\340\325J\364x\330\373-\272\266yA1\266\221\270sy8\343o\237\262\365\031&b{u\302o\253\210!\002\315\215\031 0\316x\361\2675c\037#7Yf\341p#,{Yh\320\340\313\013\252\243\324!\345\2721\350Gc\342\363v\207\037\3468\021\212}\037\200\2576\275zoH\017\2127Y\313Z\222\362\254\333\227\")\220\374\003\342\000\220+\372\311\360\035\200?O\021\034l\022\252\332R\220|\0052\214i\225)>\362\230\203\342@\233\212U[2\225\310=5\244\001 B\347\262\246s \376\273&d\007a\3416\314\325Ka\375q9H\016\310\036\023*\377\000\335\000\345^\001v\220L\260\037\000\205\346)\273\037\324\231\000\0322/I\210\352`8\234\306\233i\272\300x\222\037R\354\316v\375\t\331O\317\001\331\264\2076\313u*R\333\336\016\266\310\376h {L\360\331\302\262\035q#\376\210\007\344\027U[2\242\017\352a\330\331\324d&lMzs\260 \204\373\361\3312\375qC\261\264U\312\227\250\253\006\236\231,\362\306z\355I$\243X\005\006\032f\342\240\n\020\353D9\021\264\370\363\245\017\337\025Es\203\232b\037\022\261\020\322\205?,vxi`\305\000\306\000\342\007s\211\247\215\217SB1W\024\313m\262\304&\335\340\022\303\233\233$\324\232\364y\0361\327o\226\2259\335\230\303\237\217\253\272\337\017\035*\277Vw\261\312\014v\363|\312\370\271\331rrL6\335a\3030\316\300\305\210Pz\240\002=\317\213Tw\260\376\367\364\260\345\207&\306\311J\362\226\300\202\023\223\251\344\252\002<\357\022\323x\234\222\204\264SBW\3258NPs\"eC\303.{\321\200\035\\A\350\013\362\307\213\360\361\375{Z\r\263\243k\3435\355/w\367F\353e\211\206jqf\'h\014\316\353z\315\242\213N\320\315\207L\2210\261\340)\374\367=\343R\260\236\214z\003\214\321]K\'\003\037\202=\241\343\177J\330\344zT\256g\240\273\344I;\037V\000U\323\014vx\266\364\371\277\362\014\335\247\322N\310\273\016f\327\222\017=F\374N\373\307\257V:6q\223\3111\3276!2\235}\211\177iR\250\342\231\023\000\037\025\016\300>\003\260\017\362h\367av\376S\r\212L\n\222\336sh\3649\022d\001\200\20011\352\222\354\320\013:\352\247iLA\360\034F\205*T\307\251lRy\027~[(\\\002\320xL\245\264\2074\255\226\344r\304\374\220M\354>\331\302m\310?\246\'\343\200\354r\264\312\255\356\256d(\310\362\320\013n\3724\210\007$\327\230\007$\304@9&m\202\005\257x\223\344d\330Nf}B\200\217\256o\276`4\305\330\200\016\243\235&)C\200\322\370&\204\377\000\251\214\267EX\365t\r\304 #\320\335\217\217cR6~:\236\312\323\276\226\223\\\350\202n\376\310$j3\225W\236\235\267\010\254~x$o\016\211\213\035m\003\225\n\251B\030AZ\240\001\273\010\001\017\307;3\335\025\323\243\232jcj\231\246o\036\221|-\031\331\035m\250\302\034@F\\\261\265\230\347O\245\224\375.11\320\341\3632\233\014\335%\363#\276\2300b\000\003\347\327\0101\377\0008=\305\216%\346\\\223\014\345\336U:\2616L\223\214\214\330\347~\\\236\315\305a\330\226f\254\256\"\224]\352\342Yh\204\032x\201\257\273\216_.\321\027}\177\006:\325\251\351-\355*\231O\006\371y\245\223\350\367~\315h\303\313\313\252i\336\340S\265\366\316\323 \345\224\263!\355\236\030\036A\353\365\315\244\327Q\367!\307\343F:_n\3558\221\036\323\031U\014\353,\267wRQ\235\021\254^I\345[\024\261\310\362\321\2470\261\000Uj\325\256<j\274H\335J\316\267\246L\371\261\342\235n\014tu5L\317\263i\370NO>Z\313\\\326\023l\003\023\332T\255\\\003\017\311\272\214\365\311\264\375D\'\217\327T\321}e\315\344\344\324\263e:\322\022i=N\003\301\207\236\271\335\036F\306\245\231[\232\343\213-]993u\217J|\023\035\331I\031\326\321\274\234H\014/\332\014k\301m>o\233O<\373J\221\201\261\317ibC\372\355|\021\330\263\230\223\2653\266&\260R\304\346z\003\367\342N\325\374\202\320\335\202)\221h\243\227)\016\355\234\245B\177RN\266*\013Rc\352]\252Y>\303\256\312{\260\010%\3757\t\267O\327\022?\177\034\021V\301\230E\262\363\232\251 \376\245`\014\367a\261X\3326)M\234\245\313\36498\274q%\017\371c\237\333@v\002\265Iv\226\317\261\312@\022\007T\267\234\331L\001\200\034\002\316=5\330\373\327\"a0\r\262z\277\022\021\313\352`\232I{s\341J}\016$\002\000\225\215W\322lhO\373\270 \225 \"PT\233\207 \373\302\340\036\t\225R\251S\232\2765\037o\274\333Y\231\355D\303\220\344\347\306\317\244\333\017#\246\336\340<s\213\024\263\3437\335B|\261\304\223\2547\230\017>\021\351\005&\3453\271\3173i\024\230\373\202\210\2708\230\221\266[\304-;a`\005\250\007\\[\356&\236%J\321t^ l\016\366\324M\212[OtT\227>\312\032q\332\314\240\361\205S\213\271\365y\203\213\"\314\272|\265\357\274\232\\\334\351\372xr\233\322\220B\345\0169bq,\312N\000\006\310\003\343\342\2001ug\323\335H\331\006Xm\243\306C\246i\303\270F\025\"\316\325\037\223\030hCU\027\034\'\214&\226%#\336\2114 \0069\243\030!KuN\032\207\276\021\014\337NI\244\3523\224,\030\244\346\206\202\337^\034\216\256ZsT\017\0259\000\007\030\005cT\351A\304\212sOe\260\246d\311\362\205\2506\222\032\233\030\337W/\273\254\301\316K<\374SN\036-J\240\256\035\016J9\274\3164\351\277\343\3711[{m\345\350p\353oN\237,Lpm3\212_\242|~j\336\260\254h\230\037\330\274\310\352\227\001\020\354\271\301\334\307|0\263(e\230\304\021\000\236h\200\034P\017\233\022\242\314\374o\366Q\253-\027I\336e\236\033n\312o\311\354573\217NY]\202\204\n\333\232\225\353\324\253\033(\317\217\374o\006\031\335\244\271nXJ\330\333\367\336\222)\313\035\232\243$V\025\244\3635&lR~s\014Q\325\315\344\344\213K\037\367\303\236\372\3476\2659\335\262\n\t\272X\366\353|q\253\263\201\311\217J\256\221KR\306\231\n_\'\207g7^\322z+\2360\236j\245N_y;\225\346\017\235\033x\367\212\262L/\266}\233x\351\272\\O\352\215\226\277e3\t\262\014&\330\247\001\246}\\B\033\035I5,\376\333L!\346EO\022\024f]vh\005\234z\236\023\370 \371b!\000\003f\314\252YB\177Ru\237\3028\n\226\302\355\265D\207bL\2000\237\341\000\212\351U9\037yJF\367\001\330\344\211\177\226)\\\203\357\013\200\367\367\231V\314\330\205\177\335\373\020\034XR\3702\232\352\370\232\303-m\321\3014\2211\215\212vUD\037\324\235i\000bb\340\201\325\032X\007\344\r\277U\"&&\322\362\271\2315\203d\262\245u\272\214\211$\223\000zj$slr|\244\311\301V\035\230\360q\2305\235\0319\232\0231\262\343\006\205\257>\023\351\006{\003\004\315,\377\000y\234\322\337\320\273\331\222\260\233\020Y\232]z\333\320\207\214\r=<}\010\246o05W{\343\254\262\226@tskT\177p\214\034a\307\035m\234wL+%\210\000\342\007\030\003\363\306(\325\022\257\253\345\014\355zUKO\223{\342k\014\020\254\307\034\362\272\214\340c\020\212\030\353kj\010\243M\251\273\031@\'}\213\242N\251u\020\264\313t\236\347\341E\"\277wX\344\206\016\342\203m\217:\320\305J\300\001\030I\005\017L\363\304i\2424\341n\206!\014z\330\206i\352\214}\225M\235\363\316s3o\201\362\312\243\310B\243l:\333)\346z\272c\212in\313a\233\026\037\344\261u\265\222\036jp\317\246\322\357c\357\266J\332\310\206F\2447;\005\3102(\334\216;&\250\260hW\036\344\320{q\301\346q\243o\250\370\336|\305t\327\254\212\245\231\275I\016i\225\021\034\352>\277\037\'\303\364\236^\330\3336\2272#m\025\346\266~O\361\225-K\264\374\330\331\342\306<\271\375\0141GfNNI\254(t\013\234\346g,&\346\252\031c\253\233\031&\322j\247\344.i\244\014:\305an\201Z~\223\'^\250\352z\361\036\31291\351R\2771\370L\310B\256\207-\372\365\243\316\316lG\265P\316\225S\034\333\2037\205\007dcE\2575d\230if\251\215\261L\267\232\271\237~\337\262\337\252\261\263\217~\312f\027c\n\264\317\211\260\235\347/\027\250<\000\213\322\233\262m|\000\303?\352\333<D \225]D\005o\202\223]\267\362\"S2\256*\343qsM\263) \377\000\304\212gk\242\257\001]\327iLy\035w\317\032m\225\027\223\036\227\r\336\223)\336b\231\312\036\301\036}%1\247`\021\036\367\210Fm0{$\373\322k\262\254\277]\036\370\361\257\234y#\372qM(\313*\266\226\313\003\371bN\312C\307\215\\\371 z9\035\215\262\335L\262\372y\007\250\327\034q6\221\036\340\304j\237\0239\036\330\231KS\261\351\311\345\256\346{P\356\006SJ,W\237\006[2\363\032\214\211(\311&\323)\307\342U\007\237\020\277\"\nB\343\223&\251>\205\344\003\374&\234\010\"c_\226X\344\344u\235\251|\210\007\271\014[\207:\234\320\241\351\246\222|/m>o\233\351\310\204\022\252\r\215\206[x\2641Q\233\214B\205\225\027<x\200\215v\313\022\243N\335\352\364\323,\323\223\223\344\241\340\311\010\020\264]\3609\'g\006\024X\300\"\2065\007\013^h\305Tu\307\317\354Qi\331\246\201\220\013KG\214\216\255\216i\227\037xVa\326\3266e\245\307\020k\373\3258\200\024{\265\354\221HT:\331NV\0233\025\305\002\025\352\3139\033i\325\313PjpU\323\345jT\017\253\r\214\3159\311\3232g\262$\366\306\303\356-\026\22665\362\2651F:\234nf\342)\270\200=\265)lSz\362\213Ob-\302\033Sm1<~\236\344\366xm\226UL\317hX\333,3\216Z*\315}-\303M\264\345\013>x \244\3723J\346y\013\220$\275\222u\266\264\261\216\251\270\234Z\325}h\341\3069\217o\243\343\362\342\322\236=\314\363\313\232k\266\035>\303\241\211w\352\350NN\320\214#mS\302\242\331\272\021\013\"^jKv\331\242\233]t@\231\234\204\256l\207\266d3\214\215\214U\2264\315o\335\233<\000\233\333\034\256\315\212H \204\347\031\256\355Dq\332a\226\324\212\375\230fr\022\266)\"\354\226\335rrlm\265e\225P\021;\345\231g\266(\2042\364\370\304\345z\313\352}\032\242\364G\032p[l\366\256\227\305\021R\036rBT\257\267\013\306\362\263c7\231_s\032c$J\253\321}\246\360\231\3151\027eHz\344g^-}\250\262\'lw\240\223\225&K\364\232S\310?~\310\344\373\033\270\275\020w\346\314\277\214\310\373h\013\215}\t\321\344\314\230\377\000\006_n\007\250\336\177\252:\223\306\20723\241\357}\355\3633g\321\212\210^GC\2232)\2364.\214\352\365\326G|c\372M\260\362:\342rq\232\330\232\200\001\251-\3339l\364\304\3053\210x\022\373\346\314\330\347\366\304\302\376\241v\030\362J\321\243\332\'sR\232\3658Xz\030\303{\351\332\303\361\321\222\006Nt\257A\264y\342+\256\026u\3624d\200\303;c\020\252{q\037\036E\3313c\210\372V\223\r3L\317\277\374\032\214\345\306\222-\262\313\034\216\032\303\n\363\300V\'\267\017\036G3\'\'\034O\322\267\230[\2512fM\204\337&\373\202\025\007\027\261\222\004\345\366\000\034q\007\267\023\356\347\226\005\314r\313!\362\203\025\304\365\327\314\361b\315e\247<\340\352\274\300\014}\210w\023n\366\tU3\235$\036\346\2517\321\004\353\254l\3134\301\217@\240\007\020%bD/U\324\206\201\244Vd\3305s\342\245V\347\333\027\221\261\276Z\343\350\002\266\353\314\000\342\3545\2259\241\225\351\232\200T\316\no,r1\010\010Q\277[\000\265\034Z\343\007<Z\030\340\354T\211\306I\225\032f\226\242\351\016\203^\317slSp?(M\361\031\300PYFoC\342\n/\254\232o\232%\245\346\312_\224\210\226\\\347\003\320^\t-\035\215\265\236\316QBT0\017t#J\031\340\257\347\350C\263\327\027%m\214Od*R\346A\014m\t\026$G\226\031\227\244\345\000\240\200\372\373\220\342)\335\356\264\007\016\302\223\244*^K\201\037\'\224\262\312\027\333\276D\223\221\235\260\226=;P\004U\352\214E\013\034`\251_w\013\214\267IB\267T\324\344\231Q\375\326\247\366\202\336\021$Y\335\375\246!\256i\344\032\224x\241\255P\364\247\324\025@V\005Q\356\242\354\".\201/\214\210\352m\242\177P}\312\302\243\334\3252\351\303^}\215\213j2u\212\224q\371\245\003\217\034\316M\365.\237\032\233\207\0311\345\365\262m\"ps\313\272\273\254-&G)\233\327\252og\345\2173SP\257\201i\264\264\346j\2515\345*\250\342\346\274\326_Y\207\027j\206%&s\027\332\357b\022@Lw\\\332)\233\256\210?5\021y\260\275F\272\307\222t\305\021\250\3336w\306\256J\347?\256J\225N\301\231\344yM#}\254N\314v\370\237\035\031!\363?%\362\023\206u\010|\237061_\233\037r\367\362L$\223\267\302\261\005\371c/\'\203\030\343h\342\347NI\321\311{\217\205\362\222\027>\034\234\354\034wJ`*\343\203\263V9}\274s\247S\025|\221\263SQ\n\2333\233\257\241\326i\356\"8\362L\313\313\321b\321u*g7\\\'`F\244\236\210\316%x\352b\235\261\336\215 \215\345J\246\3223\233{\307L3#[\013\212\301\246R\232\354\245)\007\237\223\313j\375\336d\005\234\230\370\356\271)\013T\324\370\331\262\271\237\0010AHw\234\331\361\260\203\343\332\316\241\\\306\213Y\'Q\004\314\230\365J\222\334\017\345\211\311\231\354\306L\371\"\033\370\374y\311\364\254V%\243\312<R{\232\2473\356)\362\304\234uKK>\316\220\243\346\357i\312\372\334\034\030\370\237r\246i\026\236\246i\2755\332PU\200\232\272m\242\317\245\252,P\305\330x\354\234\317\222\214\376\241\025\226$\346\307\327\263\346g\3272\017>\306\330\343\226\023fZ\024\340\006\236\220\373\000\2156p\2236\2519\261\2616\035\274\370\253]\236\023s0\322\370\365\303\216\020\3631\"\231\021WWT\257\222\332\345I\224\256ic\275\231s&\330\026\206\366\354\2061cs\3410\231\264\231aJil\365X\t\r\202\203\271\0333-<\370\246`Y\024J\253\300d\307\336\222\237\323i\353*h|\021>\251\366Z\356S\033\036\rB\231\263.\271F\362I\327\2038\303\256:\325\002\037b\035N\310\254\245.I\363|\332z\225IH^\271y6(\311\325\226Rz\232~p\304\001\003\037p\021\361\007\032\251\205F\321\351\316\216e\226\311\220\211\031,\236\3252\236\274\353\025\226\325\313N\227}\256o4\030\242\262\323\354\0166S\t\266?\231\032\2236M\257\222\313d\340\204\204*\016X\342\215f\r\035\241\266@\020\252\000\352\330\265\352\010\035\241\361\343\026n4\313\312\333\252g>I\323\317\203r\342\245*H\270\247n,\234$M{\271E\214\242\205QEZ\366B\252:\365\307\312\351\342EX\360\364\373k\257#H\254\237.)\243\331Jj\244\307\313\212\367\305\350\314h\225Q\222\260\205\226\267\200UP\267$!\202\310\005b\003\037L\321\217z\004Q4\354\204\346\352\242\206\251\315\315I\367\236\016Iv<\231I\301\241S\212\032\261\323\343~\220\246svX\2643G)g\231\222\354\347\227B\200\234\360\236\217J+\315\377\000bQ\37263\255\033QT\241E\307\314\330\010\202\020\330\333X\362\270\232\030\261V<\032Y\027\233})\372\023\243/\013\334\217\236\\\323Xo-\244\330\344\312.8\\\336\\]\365\337\027\302\232Jx\276_T\330\344zm\343]\030\260\347\353.\317#\2136\203jd\212cG}\241\370\262{\224\245\214&\345\234\352#\317\263\301\245\212\366\251\262\217d\227Y\231\317P\200\237\2653p\000v\243o\026\'%\234\316e\243\006\031\211`\365\207\34158MV\275A\326\307t\265\361\207\037Y\305\301\250\366\374\366\371\272f\231-\266XT\371\374!\222zB\312b\334\246\314\345\313-\215\244]S_\210`\264\261\'{5@\364\3068\371\236u\346\323\351\364\034+F8\366a\035\345\315M\351\365\316\340\272\304\303\211\261\325\245\342h\3040\360-D\363rbQ%\357*S9^\234\322\221o\345\210\367\332\232\003\305\322\213\357Y\247\245\024\313\022\277\250\336\221\257-\244)\327\362\321\322\307m\263]p&rJ\2451\nR\352:\237:<\313]\262]g&2;\216q\341\034\003\3322\"\273N\241\346H\324\200\236f\246\311\032[=\365\315I\031\276\244\223\216\263\26528\334\254\223\022\372\317\207\301\031+\355\223\234\217}\236f\333\312\245V\367\203\255\216\261\312\026\224\276`\005\211Z*\303\307e\371\017\231\236\\j\004\243\301\215\215\253\2256&>\301:\313\024g\035\224\275+\343\217\214 E\327\264bq\261\304\314\355*\226%_\t\234\260\033\342l\004\324\237;X\261a6\212\025(\347\325\367\"\026hL\016\003\235\346\363v\361\032\014\265\262\304c\354T\000CR\266\216\234S\"\272\234\313s\231\224\236\331\220ij\266,\343\221\330\336\024*\335\000\003\347iq\"\351\204\313dC3^p\022V\302\020!Q\226%\035\210\013\311\362\343\006=P\3053\001\355JD\327k\326\034\302\304\';\321\232`\307V\'\325\016\311\314\2529eK\222\354\352\300\364\004\330\333r\246\017\211\270\304\305\256> jq\341\324\354Y\314jh\3613S\343\033\231\373bu\207\035\211\263\200a\273\245\030\364\305\212\001\014u9_2\276\252]F\322t\327jB\242\343\337\037S\021n\276\317\t\034I\3313q*\212\2460\365\246\214\320i\216\240\003\314\215\224\277\243jq\356\205\244f\31493>*\"\301\0024\347\022N%\231E\224\002\010\307\037\374\300\2060\324\307\304\207\251\2040\333\274\244\211\245\365S5\027\0377\321\003\021\026\370D\302V#$\3533\rN\n\245\024\000\000X\230\201\254\n\200\0304{\021\314\344\337\244\273Xx\235\341X\310skbT\304K3\315\036\341\322\027\270\230J=Bs\n\321\ru\000\026=]\305A\343\327\rO?\3344\333\237\313\247S\335-P-\036H\322\222\351\275+\031\030U~\363\217w+\266\020\361q4\342\334\237\242\236=;2\323Q\356tf\244\3674\331\003\324r<\236\227j!\365\013\255=\247M\010\276|s\246\237\007$d\251\254\030\323\244,\345\234\241\246T\216o\'\231\342\237O\240\370\357\217\214\321\271\\l2\312V6\322\023&\310\020\236>[sw\334\340\307\024\006\353.^\224\336\243\336\272k\264D\270\223G\236K\027R\373yx\210Y\022\304\216\330\2256\315\033i\355\206\366\323-w\341\316\251\225L\204Q\343f\302\321\266t\2537~\240j\373q\364\337\037\304\210\227\301\377\000\220s&\266\353\014\374\002/;4woo\024>W&>\325\354{Av\273\037\3432\010#~:\333{\342q\343\026y\337\246\31496jruR\2475l\314\020\247\324\344r\206\231\307\037\024Q\226\274\010\315\356Vd\345\316?\240`\006m\377\000x\343\206\\n\246(\263=\3713&\247T-\237\313\025e\377\000\\h\2437\016%~\034\263#\344\005o\215\216W\224\255\207\236\207Rq$\3077\217m\267]\244$9\251*\257\026*Ly\007\353\262\332\275\001~X\333j\355\222\353\355\030\343\244\317k\355\'n\202\213{:\271>&\226Y\027>\271\352\023\223\031\271v\3347q~?\3632G#\377\000\031\n|\236\325R\263\222\025)m\317\326`\322|\227\237S\215\363G\036k\271t\276K\223\034\314\177\217\377\000\202I-Jd\3301*k\003\354lo\204\327\311\026>g\0324\270\251\201\307\261\251L\325#16f,\004\227\274\331\347c\335\217\235V\000\364\3046>\251=\215\211M\201\0102+3\314\231\246W\323\035_`\021F\232\366?\3029g\002\021\'\266[\220}\261\2269\230\314\265\251\273\306\rs|\370h\33205\314m\236\021\352\020\241h\273\222ND\006\031\347\324\335\010b\335\216\'cfv\331\237\t\347O\257\247\220z\373C\226\034MKK=*\203\037\033D\037\3208\246M\217\001\211\256\3049\261\266.\260@\343\277W0\303L\0304\307\331\036&\342\'\010\014Y1\271\261\271a\324\251\262\3715\213\t\')d]A\0058=\343y\203\027\231\027@\231\3174\215u\220\032\2335\356\252\016.\304\222|\240\000\265\256>h*W\373(\235g\262\255i-\226\033\260d\222\353G\211\262\010T7(9\237\2244\301\325\t\247\366\004h\200\016\307\036/\256(\262=\246\014\224\272\205+\233k\254\240\232\302\342\276\370\261a\336f\317S\235X\025\375N<{3{A\223\035+m\251\276\366\332lr\243I\221t\262\372\330\177\203\216\356VW\313\0233U\233\356\347U\243\\{\214J\370\203\216/+\016I\237\247\326\374??\026\032\352ey\323\307{\322\231\276\302\225\250\202\300\211\251\006Z\304\355Z\343<\375\311\273\212\336\346\2348\274\217k~S\2137\217H\254\267I.t\207 !K7\261\241@~P\223\2118\341\331\336\n\035Q\202\241\272\"\346W\304\3631\343\261\025\362\303\345\'\034\341\267\266f\246\371W\0069\021uM\350l@_\261\271\026\214QH\355\351)\216\276\321\211V\220\325J\023!\3633\036]\n\013\272E\210\354G\225N<Z\366\301\r@\343q\375\270\317\237\343\2477\270n\343\374\204a\365-u&R\304\2637\266\220\251\261\314\216\247|+\317\216\014\374e\252\373.7?\r\247\355!\003\342_*\212g\215j\272S\227\r\243\354\366\3170&U\026\370-ig\236]b\r\264\265N\014tU$\256|R\244\214+ba-\250\371S>Pi\216:\234n\024\332\034^_\310\326%\363\225\312n|}r\275^\255\362\306\035l\260\234\241\246\014x\343\217\243\210|L\313\262<&\253\351\3273\3549\022r~\354]\020\246d~j\233fK\324\222N\260\330\365GR\016T\2312l&\346\247 \237y&\007S=\351K\346\323\220C\310\223\361\301c\312n\251sd\260\023\23213\205&S`\177\353\326\214M\253\201\222gsJ\244\213\322\242\017#\353\225,\375\255\030\rt\216Q\226\\\376\203|\313\362+2q\265\313\255\366u&K|k\332\223}\214\027\327\332\242\357\245=\317\300\224,m\216d \277\234e\363\224\273\356\352{\221\203=\266\351\374_3\301\217\304\246Q\241\301\215\2449\261\251\260!96\'tE\356\307\347F:\327jy\02182N@\312f\255{\342d\331\004\366d\222I\337=Z\225\207\023g\007/R3\232k|\344\362\017^q\226\3272uU\364\352V\322\363\340-F\251\306Ybd\361\352b\010=A<\260-\025n\273!\343\307\232[\330K;\256lz\226\305HO\\\240\233e\207\023\\\262\322\227\270 \034lQ\303GeJ\260\373\263\335\351\317P\275\304\263\256d\352\362X\300\032\217k\023\235\317\210X\354\362\361\245\302G\252}\267\315\311\336uvu\303P\000\363\342\231;$\355\256*\\\331\010SuCn\243,u\261\3314\251\364\261\367\025\277$N\0230\262L\312S)uT\347\227=y\305\235\311\344\300<@sC\362\363\"\350\022\031\205s\033bb&i\231\366\301\n\014\320\343\277\324\033YEJ\332<N>/>#i\350\363\014y\'C\033\\\251~prj\236dk\212\366\244\355\n\033\221\243\266\263Q\2254F\333\343b\013r\016\315|}\010\242\334\351\247\247k\017\307FH\331\342g\005!)LCb\251\031\361\r\342\314\223\254I\274h\003qTX\270\265\243^/\223\245\231r|M\354\217\016N\245i\2311\022|\263Fn\250\020\257$\304k\026,\250YiS\233\2461\214B\256,j\243\250\010\226Nn;\177I\342\370\234\225\372i\312=\224_dim\252Ps|=y\010\021\226\216\330\343\262\206\324\005Z\343\253\034\010\257\216v\373\nDg\215J\031K\262V\014\277N\014I\272g\204~TX4\317\363\200\035>8\003\307\000#\261\305\344zp\276C\203\025\226]\2469\2016\004>\353\227\362;bu^\256\217\346\215:\351/\236\257\356\214w\256\221u{|sss\\\323x\273\222s\221/\007\265\226Q\206\214U\000h\304A\250\215\256 T\001Kj\000{\201\327\216\247\032\321ha\344Rk\364\264\347j\014\226R\276^\\\273\2041>d\316\356\244X\214r\233\202\222\305\273N\2447\266u\203\377\000Oe\020\236=o\364\363\215\2275\177\264)d\23539\333\246\242\nU\230\327\256A\2660\271\034\334c\242R\301\246p\000\225a\2434>aQT\374el\351~~j\307\332\267{\245\367\311AI\036\007\322\263\343\352\356\031lNg\314\251j\020\033\347\200ev\342\254\237\037Z=\257\312\332\350\003\222\347\311\301\313\302i\301\314\367c\372o\203\233\026a\244Q\223?\"\327\36611wX\276!D\311\310\220p\2259\002\"\350\2053&w)\201/\006S\220\337\216\370\001\025-\321\2650\024\276)\274\251K\220\336I\201\243\302\303\023&Mu\202&q\252\362h\013\026\216\222&R\311\234\271\236\200\373kn\216\317G\036\256<bmO\021\3416\314\327 \277\251\3664\2406\363Q\014j\270I\350>\360\270\323\022\347uO\320.s\226[O}\302y\212rm\216\261;\'f\016`\242\350\223\253\022N\024\214\252\232fGY\345V\241Ff\214\222I\031vI\300<N\320\305\034\214\326\333n#:g$\315\211\210l\274\345\362\226\307\035\263\224g0\033\241\002(\245vexr\373e\344\366+\266\363lr\307#\255\024Zs\t\016!^g\267\027\235\210\360e2\225$61\330\036\271y6\304\364E\327\323\322\305\363\340v\03755^\233pbeV\004 \313,xY\224\2653\230\016K\211SO\314\202I;;\223bfD,I\225\036A\032\345\207j\3147t\000v\307\217\001\332jccsL\306\331\260!P\342ZDd\223\224Pn\351@\353\3631q\376H\250@\351Dy\261\n[8\001\306\023\311\3451\205_\027r\000\233P\000\346\300EIxslLCbl\271\352\t\317:\"\353\356\306/2\t\026L\342\251\217\306il\017?8\313ov\234\315\330\252W\354V\200\253\247\367\2279\231\265\016\023s<\364)\316P\262\307\225P?\213\023\337\211d\365\0131\316\345e\310\022\2159Q\353\"\027\3123\267\231X\335\311,\353\022N\331k\343bV\026,s2\346\324\276\227\205\212&\027d\231J\364\366\231O\367\276\214\327 !?\327\000a\221Gg\177\313{-F\252fR\346\233f\260#\246\326ZC\261\342\265\222\331z`\302\177\314\343\312\3626\216L:\204\300\2248O5U\250\213\353m\260\336\232\227\315>\372\271Fo\2439\330\371A\3151\367\035\261\235f\366\251?\344\305\000\301\315\257\273\216\276\034\221\267\310r\260L@\236\365y\251\315,\310\352\330\306\330\271z\345\355\306\023b\215b\242\362`\326\327$\240\250\002\240\325\334\032\221G\230\014x\354a\311\032r\'\034\304\257\267\')\032Pm=6\034jiB\235\035\266\001~%ky\206\342pA\264\252(\322\005\377\000\326\025\346D\274}P\352\314\324\307No\224\230\230\211g>\"\\Ofq-\256NCx0\2438\340\\\2507\212\274\312\365!\346\352uV\351\220\245\362h\247\252\307`\020\252\035A\343\002V\304\327\2251:S\304D!/s\032\227\3057d\272\210\2476e\321\016\315\255\251\270K\236\317\274\223\275y\343\217\013\014:`\340\314I}4\024X0\033\225mNj\243\333,\210xf^sd\321E\227D.\226\251q\215*b6\344\027rlN8\234\247\273Pp\225\347S\222\271\245\331\237\010<\216\247\337\254\030\246F\253\220)\032O\231\276\203\231\210?\241\325\231\352\n/\211a\322CLnJR\321s\255\3315\271\36732;\337o\215\027D\232c\2311\325Sbd)U)\310(&\330\354\214q\357m\257\304{8\217\031\020\247/`\237<\261\3635@\007k\335\213\360\327fQ\340cT\346\232\355\227\267\261\266r;\225\342\020\001\213\023\215\355\301N\335\235oM\226\0301.b\274\342\321\235\322\231\213\210\0160y\343\304\305\201\262\326\036\225S\222\031g\t\333\357\307\023\312\231\361\013D\000\342i\301\240c\251\352\222\337\275\032Blw\252\374\220<\335\337\317\001\305K\252\245/h~\256J\202I\313o\232C\366AS\265\025\006u\206&\300\216\277X8\273\036\226\245[_r\001\235\341\265S\033kRk\326\321\2268\357d\036\320E\004\220\231\250\365\\*\336\301\006G\2437\023\037\332\253\001]?\026\347vB\253\312\t\266\367\203\t\375\336D\364\224\362\212)\372y\242\226\337\001\2239\370\252\371m\251\2641-pn9\265\267\021\\\360\273\272\374^t\322\023\300w\311\271\276)#\373\336\270\363\324jI\301\271?z3\376;\263\376\336\265q:\227\224\360\246\314\277C\\\277\210p\374s\376AJ\246rg|*d\252vR-\342\233q:{i\2477\311:j\272.\246T\317\266\031\265\204Qktn\245#\'\264\266\234\250\242Y\247)\000\371e\3656_\\\215a;BU\033\203\301\361\203v\n\340\215\230rN\334\036n\010\210|\235\376\314\246\366*c\376\314\337%\223\360\342uv\3272xqi\362\365\323\217\236QB\250:\236\340\343\261\207$\351\363Y1\304I\326\223\251*f\233\346L\030\245\361\325{SA\306\022\214\223\236\017XYE\350\342\014\321ib\007\0367y\2730\350\316\214\264\336Ka\016\235\215\014\272w|\232.\322$,V\225\263j\206\204\001\371\375R\2251\2336e\321\016(\320\252\340\251u\373\361\304\332{\021N<~E\321\007\2001\251\376z\347`G#\253\366\003\026\251\261crlM\364bkx(\260\001\230\245V\323\036\331\252 \375&!\275=\377\000\247\313zM\304Qe\321\013]\030\376\265\tz<\n\363k\267M\024\310\207\266\230\226\363\310\037\366fFZ\256\262~\246\227\247\226)H\366\304\3231\353\320\250\'R\263)\352\016.\2052gg]\204\345\"\025z\036SB)\252\211\204\204\225J\2579\322\233{\2716\304\223\2533\023@c\342\206\264]\nf\016D\274\336\233HJ\226\336\335A6\313\016\344\213\321\261\000;:\036dM#\333\221\351X\246F\247;\311\371\272C\016F\214\236S\024 \007\267\\}\210\002Y\033\233\033\024\256sU`{\343\271\327Bw\273-\320\374\320\374\260E\305\341/\216\310J\227\203\244\336w\252\200\016\2008\330\365  {K\226\314~qh\221\036\371u\257\273\257\272\026$U\321\'\223]S=\236\330\246\336\342\237S\322\342|p\350\tx\036\023m\273>\251\260>\330\303\275 \n\027\271^\274y\020!\353\017T\346\311\265Xas\255\211\'\243\257T\036\310\"\350\201\036~!K\233j\033\252l\202rl}@\013\342\255\026x(\277Pm\034\242\225\261I\tT\360\214\363\321\331V\367\253\003\263\017\005\r@\226\240&J\345yU\220\273\332\022O\353\261\023\353\245V\366%\311\325.|\253\200\221\235\223\326T\217{i\236\324\332\245R\353zSyMo\025\327\367m\274u\235\255J\037\245\367\331e\310\206\3077\334\206\362q\332\262\214\342\014|^|b\344p\373\273_\037\362\021\2065/\245\224\033J\211g\006\334\345N]>\270\230\343\343\237\037\333\3502Z2G\240\335\361\275\356\362\31542l\327\007\302\t3\006\271\022NQ5} s\212\036\354\021\272\234\215\260d\343\373|\312{\221\346j3\231\027I\363{e\301r\177\2634\275\300\300=\320G\035~6h\344K\345-\306\236!\372I`S<\275\340$\316mM\'\330\230u\263\221\327t\370\233\212\365G\2148\350\315\374\020\313j\376B<r\345)\270O\333D\246T\325\030u1R\256\024DS2\272\rK\333s\234\031z\331\362\'\034MC=A\207J!\245\363\'\344h_.\331\256`GM\2242\032S2F\006\362\245V\347\303\304;\r\nd\273N@\210x\203j\225\327\234\3252[\002\"\026\313\264\353\207K\n\217Z\256\315\267\257(\374=\304Qkm}i\244\300\021\352n\375\335\343\256\200cA\203\024\354\312c\rWY\347\264\231\264]\nd\361&f\251\210Mv\260\274t\337\004SW\223\003\336\030\356\257wW7=\240\354\265\216\364_>.\2053\007\264\312\260e\371NC]sFO\253\216?j&\254\361ov\260R\253.\352\240\353\023\211\266\331K\322\037\303\353@26\256K\341i\t\\\355\317=y\337j`\307\217\356\004\020D\3670\276]\223\036\347v\313\250Fa$\222N\263\034~\360\361}h\003\023]X\324\272\271\271\245\330\022\027\221\372\305J\303\366q\"\376\211!-\247\251}ru|S\220!F[\321\350\217\331\252\017<P\350\"\263<\306\225\3152\344\333\371\347\031\226\'z\257T#\037\253\024D\0175\027\342D)T\353\323\222g\267\241\027D\007PJ36\r\273&c<\373\306[\223\262\322\251\245\347\212:?\353\256\311\3712j\233e\231\345/\214\325K+\266K\023\216$\233Ov\037\353\256~L\253\025\213\225e\357:\213h\315z\351\256\276\321W\211\231J\244\33012\254\202\215w\311\030om4V\2335\023\023\305\372\245\374\316D\237\345Q\356L\332Bi5\372hN\366jls\223\336\310csU\324\235\321\361;\021\310\346\361\374\177N\377\000\307\362\347,\373}2\220\346\324\2633i\noQ\207\014L\313\267y\364\200w\303\367\276\261\322\373\' \352\236\322\346\263|(\317\210#\342F\312\346\236$\271|\2364r!\363Zy\226fj={>Y\234\033.\013\223\375\231\245\356\006P\367A\216\377\000\016\377\000\233\017\231\344\327\361%\022R|_2\347\324\031\300\362\225QL\312\3509Ki\177\231\252\3641\247OfO\013\035S%\332\177\366CJfMG?\252\376X\233\323\035\024\371W\033N!R\254\345J\253xyC\304\231\'*\236f\326\251A\263^\274\353\036\250\275\330\373\001\254>\314W\\{j\265\364\325j{\336\246f$\327f\313\212\362\023\352lN\263\366\005\027\307\037j-\233Hc\254\234\370\331\2656\036G\241\214mF\320$R\232\002\267$\314\346*\016\240=U\332\001\325\235VlG\325\316\212PL\316WyS\323\247&\333\322\017\024\250\010\302\003\360bgW\327<\271\3502$\362ec\343\343\304\366\362V\0239\355\211z\365\t-\272\242\306?xp\332\231F\333G\375\344\303\273\372\004v(\375J\243\257\355D\342P\323\263<\325\343\265\312\265\367\0132I$\355^8\361=\252\261tI\247\032H\231\257Jpg\224\035lw\262\037\200Q\016\306\221\345.\27011\r\211\270A:\356\216\245_z\035\215 i\217\302jn\251r\347\333X\305\274N\035\271\366\215\031sF8^R\004\252\231*k\312\253\003\327~\2649\2721\372\'\305p\'\203X\333\3459\374\315\331?\000.\261\330\227.!\345/\371\264S+\242\024\315-JLS\312c\363\222\020>o+\t\367\r\343\007\237\270\216/\313\374U\"\277\253\257\361\234\353\314\376\314\256\261\"\246\325\'\245sK`y\032\350\370l\234|\330m\372\303\351\277\352\315\036\344\264\312\223Et\341\346\247\275\'\226\370\277\364y\305\305y\2475?\244q\370\247\373y3\212\246\3057\224\274\036)\334\304-\214q\023\266\355\357Q\247;\312b\022\251U\034\216V)\311/\253\370\276Lb\2156\3629\201\263\006\236\346\245I\004\020\234\223\0168\356H\260\002\260\343\316-\274\266_\315\230\307W\313\272i\237?\265Y\331\326fsK\220Pu\2122y$`\325\003\316\253\355\212>\253\213\307\364\370~nx\233zTNL\n\223gIT\372\035\363\363E\263\014\2253\234b\234\206\023l>\301G\243\265\212f\027A\3712\346\313\261\t\256\307\240\375q\342\364D\201\231,\007\224\244\210!\020\000dyT\027D4\017{L\227\203\023\037Hjss\324Z#m\377\000O\273\037lX\235\230\362\221\251A\241\323N/\215\233M\201\344t\321\266\223\250\016\250\351\031\211Nj\346\331\020\354u-L\235D\023\207\220\220\177l\277w\022\035\216\254\030H\023)\3416\021\304t\016\240\274\245\341P\016\262\252\273\322\234\353\203\345\216\354E(\021\205|wy\275Xe\255\272Cx\235\237\226\001\3411\230u\266\355y\310[\031lw\311\020\354\362R\031{9r\275}\217V\017\327\265\016\312d\3255.J\245M\326\365\277}\346\342\'\022\237T$nI[\036\317\337\377\000&\204]\022u2L\223\032\24757\245?\255(\216\316\246\027\271\251R\224\327\257\'&\033:\244\362\032K\256\315\257Q\277o\230\373\200sc\357>\037\207^4\355\300\346f\231\205\361)%T\2251\021\365\031rD\306\241\302\266\017,\373<=\256\315\242R\204B\022\3670]\177\006)\225\321\0100\017s\236^\360cb\253\016X\355\354\242\376(\3115\276O\344\333|\224\307\374W\004\275D\222:^\341\n\024\261\220\275i\004\331_\026\022\003\014\374\261\\\361\360\307\362\206\033\362\363O\252\312N\261\215\216\355v\300hl:\220|\261\267\3610\332>\231\355\314\315?\332\265\236h\226\212\237\023\037\342\"\032W\021\2776\346\376\2704\005\352G3\223\361\270m?M\334~V]}\250\t\332\204\246f?\031\261\367H}C\313#\326v\311\371+\307\314s~61G\247\320\340\347\316I3\321\324\324\347(=\221\327}\221\221\363Y1DK\273\203\2211\r{2w\301*UBn\254I\224\345\327\222[q\335X\364\375\232\321\217\3438\376\335?\223\345\314\306\231\312\367\037U\333\307\017\225\353\345\235\270\203\177S\277\307\223\005MNFg7\237\'\327t\245\216)\230]\005\215\"U9\252\257BwG\036\242\000\223\024\246\315\240<7U1\004\342\017t{%9\322\034\332C\036\361\256Xw$\237w\332\371\240\272!\263\321\245L\330\332Ccb[\002\023\223bI<\221`\320\213\265\246`\307\r*\244\307\362\351\367\230\363z\r\266\377\000\357\356D{.\352jujsU\364\024\336\271\244\373\033\034\211 3)\307\321\257\016\307V~L\205\261Tq\332\204\201\'\326\255\340;J\253\260c\334%\001+\322\266e\334\374\237#\326\327\343\3056\035\244\226\245*[OK\274\'8\275w>\266?\253\036\242\234\2571+\033i\372\213u\004\226I$\362E\200\032\034\341@V\347*\300Jo9\013}\373\254\211$\207\254]\234\237\000\316\244\276\346\323\026S\325\036b\364dR\005.jHL\226<\342a\234\227\364\367%\342\027L\222\205.C\177\273\374\030\261\372o\032\272\227\311g\\\010\306\231*m\232\302:}\265\014\2253\275\256\315\242\252\316\241\014t\366\247\347\t\217\203^c.|\221\016\226:zL\350A\017\213HS\277\257\313|\261>:\216U\275/\204\000\272\246\275F\3138\323]\311\251\355\361-\330\376_\365R\'\213\3228\351\355QM\256\252R\251\315\243\334\227\323\245\216\236\221$sR\246\3051\227\313\023\032h\267\032a\351\221\014\241<\246\274\271\246\260u?\371\2222@Y\234\332\340\3204>\334r\271\\?4\355\267\217\223\303\032\224$\342\337%\224\336\014\276p\215\215a:\265?\233\233\037#\233\342-\301\366\355c\346G,\032g.\013\021\307\232~\245\034\270\372{\203\220\025\304\266\363\250e\206po(&\033:\270\201^lBhI\324\205+\276\253\227\212d\352\354\215\251S\232\222\022\357\3525$\362\246B\211KTQ,\226\226Fd\273d/\3122\313\016\370<\320F\312)\262l5PN\246\245&A}C\014\310\213\307\201\001\233,>\265\3661\304k\022Ij\272\003\340\033T\256J\225Nk\t@\360\332F\035\331r\373\361\337\004Sa$\225R\335[W*\362\213O|?$z\210\371\251\n[\311\371\326^\307\340\307\200\255\036\314K\302\242I!\340\231\322\246\267\372\304\000xs\t\354\261/\252\225\364=\220\031\315\346\353\263\376$v\376\017\014d\267\266.E\346%jI\206f\337\257\327\365G\335a\256\237?\235?\033\214[{i\222\250\224\3030k\3426\235CN:{R\323\232\273\312\234\327\204~$q\271Y&%\322\307OM\003E\303J\330\332B_\'\'\340\216\267\035\305\317m\254\343\235S]\243e\231+]\240s\013\307\343E\221\351<t\366\253\237\237\363\222>\347\344\214\\\213\352],t\364\207\251r\341:\217\326<s\257\226h\337\213Y\004\243r\372\324h\301\310\333\314\270t\220\341\306\3074\3301\3151\007\220\243\365\331\024O6:\363\252\311\027\236\024\240s\2032\226\'+\312e6\350To\337\255\327\353\314\371.\177\013\361m\351\332\342f\374\270\366\014\225\321\314\333OWc\217\206\316\244)>\363\027I\324\262W)M\024\311\3250\241\233\323\344\355y\340-\004\230w\244\036(>(Q\\\265*\005y\264l\242\233<r\270\'P\303>\013\352\360\014\210\274,\006@f@\013\273\373c\211\221\254\013\253\312\263\201b.\357s\366C\030j\031\235\330\235\223\261\332R{V\3263\316M\373;\235\330\246\312l\265\245\343;\275\304v?\341c\377\000\367\366\304D^pwTI\247\256\017w+\376\330\n=\342bu9N2\216\354Z\030\206.\354\007\200`\221)\310~\350\256\237\311\344},\206\243{\2705\273\367me\332\031\376\376\357\357\371{\221\366\337\023\374\\\336O\332\301\225N\026/E\374#\267\205\306\314uruS\376\330\273\"\232 \323#\222\217\366\367#&\177\246\254\177h\025\260\260\332\036\273\340\216\006\177\267O\037\322\370\223\316\025\333\370o%\373\361\365|w\027:d5\307\335\277\214l\263\025~\320\251\221\305G\355\376?\254h\257\372j\306\257\036\305\335\264T\203z7\370\367#\233\311\373t\261\375#`\027v\306\333\374j\326\364\225\277e\177\370\307.\262\320)\250\341*R\032\275\313+\"\177u^\314]\022\016$]\333\033o\361\213\361\252\023[\272\271\250\362O\375\375\310d\025\362c\004J\234^\177\263\374#\342]\263\225\270\240?\000.\354%\345B9\036*\226?\341\024\331t4\207{\303cZ9P\200\226\204\035\323\\\0045&\233\373\177\177v\257s\366w\003\377\000\010\350\341s\326\302\224\205\223\243\335\213\262\000+\367ag\225~Z\212(\262\372\224\001wc\327\205\200]\330\017\377\331"
      }
    }
  }
  feature {
    key: "image/filename"
    value {
      bytes_list {
        value: "raccoon-160.jpg"
      }
    }
  }
  feature {
    key: "image/format"
    value {
      bytes_list {
        value: "jpg"
      }
    }
  }
  feature {
    key: "image/height"
    value {
      int64_list {
        value: 197
      }
    }
  }
  feature {
    key: "image/object/bbox/xmax"
    value {
      float_list {
        value: 0.6328125
      }
    }
  }
  feature {
    key: "image/object/bbox/xmin"
    value {
      float_list {
        value: 0.02734375
      }
    }
  }
  feature {
    key: "image/object/bbox/ymax"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "image/object/bbox/ymin"
    value {
      float_list {
        value: 0.2131979763507843
      }
    }
  }
  feature {
    key: "image/object/class/label"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "image/object/class/text"
    value {
      bytes_list {
        value: "raccoon"
      }
    }
  }
  feature {
    key: "image/source_id"
    value {
      bytes_list {
        value: "raccoon-160.jpg"
      }
    }
  }
  feature {
    key: "image/width"
    value {
      int64_list {
        value: 256
      }
    }
  }
}